## Notebook for spike sorting from .kwd data using the Pipeline of mountainsort (linux channel has pipeline)
Uses:
    - intan2kwik (https://github.com/zekearneodo/intan2kwik/blob/master/README.md)
    - mountainlab suite(https://github.com/flatironinstitute/mountainlab-js)
    - mountainsort https://github.com/flatironinstitute/mountainsort_examples/blob/master/README.md
    - mountainsort examples https://github.com/flatironinstitute/mountainsort_examples/blob/master/README.md

In [1]:
import socket
import os
import glob
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import subprocess
from datetime import timedelta
from importlib import reload

# pipeline imports
from pipefinch.neural.convert import intan
from pipefinch.neural.sort.mountain import core as msc
from pipefinch.h5tools.kwik import kutil
from pipefinch.pipeline import probes


from pipefinch.h5tools.kwik import kwdfunctions as kwdf

from intan2kwik import kwd

#mountainsort imports (for sorting)
#import mountainlab_pytools.mlproc as mlp

import logging

# Setup the logger
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

ch = logging.StreamHandler()
ch.setLevel(logging.INFO)
ch.setFormatter(formatter)
logger.addHandler(ch)
        
logger.info('Logger set')
logger.info('Hostname {}'.format(socket.gethostname()))

2019-07-29 15:58:54,641 - root - INFO - Logger set
2019-07-29 15:58:54,642 - root - INFO - Hostname zpikezorter


### Session parameters and raw files

In [44]:
from pipefinch.pipeline import filestructure as et
reload(et)
reload(kwd)

sess_par = {'bird': 'b11k10',
           'sess': '2019-07-11_0250_01',
           'probe': 'probe_0', # probe to sort ('probe_0', 'probe_1') (to lookup in the rig_par which port to extract)
            'sort': 1
           }

exp_struct = et.get_exp_struct(sess_par['bird'], sess_par['sess'], sess_par['sort'])

# mountainsort parameters
sort_params = {'adjacency_radius': -1,
              'detect_threshold': 2,
              'freq_min': 600}

# differetn mountainsort parameters
ds_params = {'detect_sign': -1}


# convenient paths
kwik_folder = exp_struct['folders']['kwik']
msort_folder = exp_struct['folders']['msort']
raw_folder = exp_struct['folders']['raw']
kwd_path = exp_struct['files']['kwd']
bin_path = exp_struct['files']['mda_raw']

In [45]:
exp_struct

{'folders': {'raw': '/mnt/microdrive/birds/b11k10/Ephys/raw/2019-07-11_0250_01',
  'kwik': '/data/experiment/microdrive/b11k10/Ephys/kwik/2019-07-11_0250_01',
  'msort': '/data/experiment/microdrive/b11k10/Ephys/msort/2019-07-11_0250_01',
  'ksort': '/data/experiment/microdrive/b11k10/Ephys/ksort/2019-07-11_0250_01'},
 'files': {'par': '/data/experiment/microdrive/b11k10/Ephys/ksort/2019-07-11_0250_01/params.json',
  'set': '/mnt/microdrive/birds/b11k10/Ephys/raw/2019-07-11_0250_01/settings.isf',
  'rig': '/mnt/microdrive/birds/b11k10/Ephys/raw/2019-07-11_0250_01/rig.json',
  'kwd': '/data/experiment/microdrive/b11k10/Ephys/kwik/2019-07-11_0250_01/streams.kwd',
  'kwik': '/data/experiment/microdrive/b11k10/Ephys/kwik/2019-07-11_0250_01/sort_1/spikes.kwik',
  'kwe': '/data/experiment/microdrive/b11k10/Ephys/kwik/2019-07-11_0250_01/events.kwe',
  'mda_raw': '/data/experiment/microdrive/b11k10/Ephys/msort/2019-07-11_0250_01/raw.mda',
  'bin_raw': '/data/experiment/microdrive/b11k10/Ephys/

### convert the whole session to a .kwd file
Conversion sends every .rhd file in the folder to a rec in the .kwd file (experiment.kwd in the session ss folder)
All of the files and all of the channels are converted; filtering and subselection of sub-epochs and channels occurs later.
The .kwd is raw data, only in a friendlier format.


#### Make a file for the session for the first time

In [46]:
reload(kwd)
## Convert the whole session to a kwd file
os.makedirs(kwik_folder, exist_ok=True)
first_intan_hdr, sess_pd = kwd.intan_to_kwd(raw_folder, kwd_path, overwrite=False)

2019-07-29 22:59:26,485 - intan2kwik.kwd - INFO - reading intan chans data across all of rec /mnt/microdrive/birds/b11k10/Ephys/raw/2019-07-11_0250_01
2019-07-29 22:59:26,546 - intan2kwik.kwd - INFO - Found 688 .rhd files split in 10 recordings
2019-07-29 22:59:26,547 - intan2kwik.kwd - INFO - dest file: /data/experiment/microdrive/b11k10/Ephys/kwik/2019-07-11_0250_01/streams.kwd


2019-07-29 23:18:51,214 - intan2kwik.kwd - INFO - moving back to /data/experiment/microdrive/b11k10/Ephys/kwik/2019-07-11_0250_01/streams.kwd


2019-07-29 23:19:48,960 - intan2kwik.kwd - INFO - removing temp file


#### Update a session with subsequently recorded rhd files

In [47]:
reload(kwdf)
_, nu_pd, _ = kwdf.update_kwd(kwd_path, raw_folder)

2019-07-30 00:02:41,040 - pipefinch.h5tools.kwik.kwdfunctions - INFO - updating kwd file /data/experiment/microdrive/b11k10/Ephys/kwik/2019-07-11_0250_01/streams.kwd from folder /mnt/microdrive/birds/b11k10/Ephys/raw/2019-07-11_0250_01
2019-07-30 00:02:41,161 - pipefinch.h5tools.kwik.kwdfunctions - INFO - No new files to add to the file


## Make .mda file with a set of recordings in a session
 - pick all in port A
 - get all rec within a time range


In [49]:
# get the session meta
pd_meta = kwdf.get_all_rec_meta(kwd_path)
pd_meta.head(1)


,bit_depth,name,sample_rate,start_sample,start_time,channel_bit_volts,channel_names,channels_sample_rate,dig_channel_names,is_multiSampleRate_data,valid_samples,samples_count
0,16,0,20000.0,0,2019-07-11 09:25:20,"[0.195, 0.195, 0.195, 0.195, 0.195, 0.195, 0.1...","[A-003, A-004, A-005, A-006, A-007, A-008, A-0...","[20000.0, 20000.0, 20000.0, 20000.0, 20000.0, ...","[DIN-00, DIN-01]",0,"[1200240.0, 1200240.0, 1200240.0, 1200240.0, 1...",1200240


### pick a time interval of the recordings

In [50]:
def select_time_span(meta_pd, start, span_minutes):
    end = start + timedelta(minutes=span_minutes)
    pd_selection = meta_pd.loc[meta_pd['start_time'].between(start, end)]
    return pd_selection

pd_meta_selection = select_time_span(pd_meta, pd_meta['start_time'][0]+ timedelta(minutes=0*12), 360)

In [51]:
pd_meta['start_time'].max() - pd_meta['start_time'].min()

Timedelta('0 days 10:49:42')

In [52]:
# for instance
pd_meta_selection.index

Int64Index([0, 1, 2, 3, 4], dtype='int64')

In [14]:
# load the rig parameters
rig_par_file = exp_struct['files']['rig']
with open(rig_par_file, 'r') as fp:
    rig_par = json.load(fp)

# get the probe and the port where the probe was connected
selected_probe = sess_par['probe']
probe_port = rig_par['chan']['port'][selected_probe].strip('-')

# get the channel indices of the probe's port
wanted_chans = np.array([probe_port + '-'])  # all ephys channels

chan_list = kwdf.get_all_chan_names(pd_meta_selection, chan_filt=wanted_chans)

#all_rec_list = kutil.get_rec_list(exp_struct['files']['kwd'])
selection_rec_list = np.unique(pd_meta_selection['name'])

rec_chans = pd_meta.loc[pd_meta['name'] == selection_rec_list[0], 'channel_names'].values
rec_chans_idx = kwdf.find_chan_names_idx(rec_chans[0], chan_list)

# make the mda binary file
bin_path = exp_struct['files']['mda_raw']
os.makedirs(exp_struct['folders']['msort'], exist_ok=True)
bin_file = kwdf.kwd_to_binary(exp_struct['files']['kwd'],
                              exp_struct['files']['mda_raw'],
                              chan_list=chan_list,
                              rec_list=selection_rec_list, header='mda')


2019-07-29 16:10:47,014 - pipefinch.h5tools.kwik.kwdfunctions - INFO - Writing kwd_file /data/experiment/microdrive/b11k10/Ephys/kwik/2019-07-08_0250_02/streams.kwd to binary
2019-07-29 16:10:47,289 - pipefinch.h5tools.kwik.kwdfunctions - INFO - Channels to extract: ['A-003' 'A-005' 'A-006' 'A-007' 'A-008' 'A-009' 'A-012' 'A-014' 'A-017'
 'A-018' 'A-021' 'A-024' 'A-025' 'A-026' 'A-027' 'A-028' 'A-029' 'A-030']
2019-07-29 16:10:47,290 - pipefinch.h5tools.kwik.kwdfunctions - INFO - Will go through recs [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121

2019-07-29 16:11:06,576 - pipefinch.h5tools.kwik.kwdfunctions - INFO - 2051464320 elements written


## Scripts for sorting

### prep the files with their nice formats, locations and names


In [21]:
reload(msc)
file_paths, out_folder = msc.make_paths(exp_struct['folders']['msort'])
os.makedirs(exp_struct['folders']['msort'], exist_ok=True)

# make the probe file
rec_chans = pd_meta.loc[pd_meta['name']==0, 'channel_names'].values
rec_chans_idx = kwdf.find_chan_names_idx(rec_chans[0], chan_list)
probe = rig_par['probe'][selected_probe]['model']
headstage = rig_par['probe'][selected_probe]['headstage']
probe_chans = rec_chans_idx - np.min(rec_chans_idx)
# try to make a probe. If it is not possible, force adjacency_radius to -1.
try:
    probe_geom = probes.make_map(probe, probe_chans, headstage)
    np.savetxt(file_paths['geom'], probe_geom, delimiter=',')
except KeyError as err:
    logger.warning('Probe could not be made, probe or headstage not found {} in probes.py. Will sort with no geometry'.format(err))
    sort_params['adjacency_radius'] = -1

# parameters to pass to the msort scripts
ds_params.update({'samplerate': int(kwdf.get_sampling_rate(pd_meta, 0)), # required
            })


with open(file_paths['params'], 'w') as fp:
    json.dump(ds_params, fp)
    logger.info('Created session par files {}'.format(file_paths['params']))


2019-07-29 16:21:16,304 - root - WARNING - Probe could not be made, probe or headstage not found 'microdrive-8' in probes.py. Will sort with no geometry
2019-07-29 16:21:16,308 - root - INFO - Created session par files /data/experiment/microdrive/b11k10/Ephys/msort/2019-07-08_0250_02/params.json


In [22]:
file_paths

{'mda': '/data/experiment/microdrive/b11k10/Ephys/msort/2019-07-08_0250_02/raw.mda',
 'params': '/data/experiment/microdrive/b11k10/Ephys/msort/2019-07-08_0250_02/params.json',
 'geom': '/data/experiment/microdrive/b11k10/Ephys/msort/2019-07-08_0250_02/geom.csv',
 'filt': '/data/experiment/microdrive/b11k10/Ephys/msort/2019-07-08_0250_02/sort_out/filt.mda.prv',
 'pre': '/data/experiment/microdrive/b11k10/Ephys/msort/2019-07-08_0250_02/sort_out/pre.mda.prv',
 'firings': '/data/experiment/microdrive/b11k10/Ephys/msort/2019-07-08_0250_02/sort_out/firings.mda',
 'firings_curated': '/data/experiment/microdrive/b11k10/Ephys/msort/2019-07-08_0250_02/sort_out/firings_curated.mda',
 'cluster_metrics': '/data/experiment/microdrive/b11k10/Ephys/msort/2019-07-08_0250_02/sort_out/cluster_metrics.json',
 'cluster_metrics_curated': '/data/experiment/microdrive/b11k10/Ephys/msort/2019-07-08_0250_02/sort_out/cluster_metrics_curated.json'}

In [23]:
# test mountainsort method for reading sorting parameters
msc.read_dataset_params(exp_struct['folders']['msort'])

{'detect_sign': -1, 'samplerate': 20000}

In [33]:
from mountainlab_pytools import mlproc as mlp
Pipeline=mlp.initPipeline()

JSProxyWidget(status='Not yet rendered')

In [31]:
sort_params = {'adjacency_radius': -1,
              'detect_threshold': 2,
              'freq_min': 600}

In [32]:
output_dir = os.path.join(exp_struct['folders']['msort'], 'sort_out');

with Pipeline:
    msc.sort_dataset(file_paths=file_paths, **sort_params);

2019-07-29 16:29:20,547 - pipefinch.sort.mountain.comre - INFO - Bandpass filter


RUNNING: ml-run-process ephys.bandpass_filter --inputs timeseries:/data/experiment/microdrive/b11k10/Ephys/msort/2019-07-08_0250_02/raw.mda --parameters freq_max:6000 freq_min:600 samplerate:20000 --outputs timeseries_out:/data/experiment/microdrive/b11k10/Ephys/msort/2019-07-08_0250_02/sort_out/filt.mda.prv


2019-07-29 16:30:11,149 - pipefinch.sort.mountain.comre - INFO - Whitening


[ Getting processor spec... ]
[ Checking inputs and substituting prvs ... ]
[ Computing process signature ... ]
Process signature: 55936ee309e68fa779bf38d998755b8ced7d97ba
[ Checking outputs... ]
{"timeseries_out":"/data/experiment/microdrive/b11k10/Ephys/msort/2019-07-08_0250_02/sort_out/filt.mda.prv"}
Processing ouput - /data/experiment/microdrive/b11k10/Ephys/msort/2019-07-08_0250_02/sort_out/filt.mda.prv
false
{"timeseries_out":"/data/experiment/msort-tmp/output_55936ee309e68fa779bf38d998755b8ced7d97ba_timeseries_out.mda"}
[ Checking process cache ... ]
[ Creating temporary directory ... ]
[ Creating links to input files... ]
[ Preparing temporary outputs... ]
Processing ouput - /data/experiment/msort-tmp/output_55936ee309e68fa779bf38d998755b8ced7d97ba_timeseries_out.mda
false
[ Initializing process ... ]
[ Running ... ] /home/ezequiel/anaconda3/envs/sort/bin/python3 /home/ezequiel/anaconda3/envs/sort/etc/mountainlab/packages/ml_ephys/preprocessing/preprocessing.py.mp ephys.bandpas

2019-07-29 16:31:01,635 - pipefinch.sort.mountain.comre - INFO - Sorting


...

[ Getting processor spec... ]
[ Checking inputs and substituting prvs ... ]
[ Computing process signature ... ]
Process signature: 517f3cbd426f177d465329e4e1c09d55d057b8ac
[ Checking outputs... ]
{"firings_out":"/data/experiment/microdrive/b11k10/Ephys/msort/2019-07-08_0250_02/sort_out/firings.mda"}
Processing ouput - /data/experiment/microdrive/b11k10/Ephys/msort/2019-07-08_0250_02/sort_out/firings.mda
false
{"firings_out":"/data/experiment/microdrive/b11k10/Ephys/msort/2019-07-08_0250_02/sort_out/firings.mda"}
[ Checking process cache ... ]
[ Creating temporary directory ... ]
[ Creating links to input files... ]
[ Preparing temporary outputs... ]
Processing ouput - /data/experiment/microdrive/b11k10/Ephys/msort/2019-07-08_0250_02/sort_out/firings.mda
false
[ Initializing process ... ]
[ Running ... ] /home/ezequiel/anaconda3/envs/sort/bin/python3 /home/ezequiel/anaconda3/envs/sort/etc/mountainlab/packages/ml_ms4alg/ms4alg_spec.py.mp ms4alg.sort --_tempdir=/data/experiment/msort-tmp/

2019-07-29 16:46:54,685 - pipefinch.sort.mountain.comre - INFO - Automatically curating


[ Getting processor spec... ]
[ Checking inputs and substituting prvs ... ]
[ Computing process signature ... ]
Process signature: cceae8253cbcb2b24f06e9de7c6a0585748a9aaf
[ Checking outputs... ]
{"metrics_out":"/data/experiment/microdrive/b11k10/Ephys/msort/2019-07-08_0250_02/sort_out/cluster_metrics.json"}
Processing ouput - /data/experiment/microdrive/b11k10/Ephys/msort/2019-07-08_0250_02/sort_out/cluster_metrics.json
false
{"metrics_out":"/data/experiment/microdrive/b11k10/Ephys/msort/2019-07-08_0250_02/sort_out/cluster_metrics.json"}
[ Checking process cache ... ]
[ Creating temporary directory ... ]
[ Creating links to input files... ]
[ Preparing temporary outputs... ]
Processing ouput - /data/experiment/microdrive/b11k10/Ephys/msort/2019-07-08_0250_02/sort_out/cluster_metrics.json
false
[ Initializing process ... ]
[ Running ... ] /home/ezequiel/anaconda3/envs/sort/etc/mountainlab/packages/ms3.mp ms3.combine_cluster_metrics --_tempdir=/data/experiment/msort-tmp/tempdir_cceae825

Output()

Finished pipeline.


In [34]:
msc.sort_dataset?

Signature:
msc.sort_dataset(
    *,
    file_paths:dict,
    freq_min:int=600,
    freq_max:int=6000,
    adjacency_radius:int=1,
    detect_threshold:float=3,
    dispatch_method:str='run',
    opts:dict={},
    no_auto_metrics=False,
)
Docstring: <no docstring>
File:      ~/repos/pipefinch/pipefinch/neural/sort/mountain/core.py
Type:      function


In [24]:
file_paths

{'mda': '/mnt/scratch/experiment/p14r14/Ephys/msort/2019-02-14_2250_01/raw.mda',
 'params': '/mnt/scratch/experiment/p14r14/Ephys/msort/2019-02-14_2250_01/params.json',
 'geom': '/mnt/scratch/experiment/p14r14/Ephys/msort/2019-02-14_2250_01/geom.csv',
 'filt': '/mnt/scratch/experiment/p14r14/Ephys/msort/2019-02-14_2250_01/sort_out/filt.mda.prv',
 'pre': '/mnt/scratch/experiment/p14r14/Ephys/msort/2019-02-14_2250_01/sort_out/pre.mda.prv',
 'firings': '/mnt/scratch/experiment/p14r14/Ephys/msort/2019-02-14_2250_01/sort_out/firings.mda',
 'firings_curated': '/mnt/scratch/experiment/p14r14/Ephys/msort/2019-02-14_2250_01/sort_out/firings_curated.mda',
 'cluster_metrics': '/mnt/scratch/experiment/p14r14/Ephys/msort/2019-02-14_2250_01/sort_out/cluster_metrics.json',
 'cluster_metrics_curated': '/mnt/scratch/experiment/p14r14/Ephys/msort/2019-02-14_2250_01/sort_out/cluster_metrics_curated.json'}

## Command for viewing:
 - open up terminal with the environment msort
 - go go the ss_data folder for the session
 - run the command: qt-mountainview --raw raw.mda --filt sort_out/filt.mda.prv --pre sort_out/pre.mda.prv --samplerate=20000 --firings sort_out/firings.mda --cluster_metrics sort_out/cluster_metrics.json

# After manual curation
 - save the curated spikes in the sort_out/firings_curated.mda
 - come back to the notebook and run 

In [38]:
from pipefinch.h5tools.kwik import kwikfunctions as kwkf
reload(kwkf)
reload(et)
firings_to_save = 'firings_curated' # 'curated' or 'firings' for default_output

metrics_to_save = 'cluster_metrics_curated' if firings_to_save == 'firings_curated' else 'cluster_metrics'
kwkf.mda_to_kwik(exp_struct['files']['kwd'],
                 exp_struct['files']['kwik'],
                 file_paths[firings_to_save],
                file_paths[metrics_to_save], 
                rec_in_binary=selection_rec_list)


2019-07-29 17:44:11,672 - pipefinch.h5tools.kwik.kwikfunctions - INFO - Creating kwik file /data/experiment/microdrive/b11k10/Ephys/kwik/2019-07-08_0250_02/sort_1/spikes.kwik from firings /data/experiment/microdrive/b11k10/Ephys/msort/2019-07-08_0250_02/sort_out/firings_curated.mda
2019-07-29 17:44:15,428 - pipefinch.h5tools.kwik.kwikfunctions - INFO - 20000.0


In [39]:
selection_rec_list

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [40]:
### extract all unit waveforms
from pipefinch.neural import units
reload(units)
units.get_all_unit_waveforms(exp_struct['files']['kwik'], exp_struct['files']['kwd'], port=probe_port)

2019-07-29 17:44:20,965 - pipefinch.neural.units - INFO - About to get all waveforms for 31 units in file /data/experiment/microdrive/b11k10/Ephys/kwik/2019-07-08_0250_02/sort_1/spikes.kwik


0

#### cleanup the intermediate sort files and the mountainsort .mda files

In [124]:
reload(et)
reload(msc)
et.msort_cleanup(exp_struct)
msc.clean_tmp_dir()

2019-03-08 21:31:49,094 - pipefinch.pipeline.filestructure - INFO - removing (if exists) msort mda file /mnt/scratch/experiment/o3g3_01/Ephys/msort/2019-02-10_2500_01/raw.mda 
2019-03-08 21:31:49,601 - pipefinch.sort.mountain.comre - INFO - Cleaning up msort temp dir /mnt/scratch/experiment/mountainlab-tmp


In [165]:
 exp_struct['files']

{'par': '/media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-13_1750_01/params.json',
 'set': '/mnt/zuperfinchjr/Data/p14r14/ephys/raw/2019-02-13_1750_01/settings.isf',
 'kwd': '/media/zinch/Windows/experiment/p14r14/ephys/kwik/2019-02-13_1750_01/streams.kwd',
 'kwik': '/media/zinch/Windows/experiment/p14r14/ephys/kwik/2019-02-13_1750_01/spikes.kwik',
 'kwe': '/media/zinch/Windows/experiment/p14r14/ephys/kwik/2019-02-13_1750_01/events.kwe',
 'mda_raw': '/media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-13_1750_01/raw.mda'}